In [1]:
from utilities.proj1_helpers import load_csv_data
import numpy as np
from utilities.pca import compute_pca
from utilities.preprocessing import standard_scaler

In [2]:
train_data = load_csv_data("data/train.csv")
y, x, ids = train_data[0], train_data[1], train_data[2]
y[np.where(y == -1)] = 0
N, D = x.shape
print(y.shape, x.shape, ids.shape)

(250000,) (250000, 30) (250000,)


In [3]:
x = standard_scaler(x)
x_reduced, vectors = compute_pca(x)

In [7]:
def sigmoid(t):
    """apply sigmoid function on t."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    z = np.exp(t)
    return z/(1 + z)

def calculate_loss(y, tx, w):
    """compute the cost by negative log likelihood."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    z = tx.dot(w)
    neg_log_L = np.sum(np.log(1 + np.exp(z))) - y.T.dot(z)
    return neg_log_L[0]

def calculate_gradient(y, tx, w):
    """compute the gradient of loss."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # TODO
    # ***************************************************
    z = sigmoid(tx.dot(w))[:, 0] - y
    grad = tx.T.dot(z)
    return grad

def calculate_hessian(y, tx, w):
    """return the hessian of the loss function."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # calculate hessian: TODO
    # ***************************************************
    z = sigmoid(tx.dot(w))
    S_nn= z*(1-z)
    H = tx.T.dot(S_nn*tx)
    return H

def logistic_regression(y, tx, w):
    """return the loss, gradient, and hessian."""
    # ***************************************************
    # INSERT YOUR CODE HERE
    # return loss, gradient, and hessian: TODO
    # ***************************************************
    loss = calculate_loss(y, tx, w)
    grad = calculate_gradient(y, tx, w)
    H = calculate_hessian(y, tx, w)
    return loss, grad, H

def learning_by_newton_method(y, tx, w):
    """
    Do one step on Newton's method.
    return the loss and updated w.
    """
    # ***************************************************
    # INSERT YOUR CODE HERE
    # return loss, gradient and hessian: TODO
    # ***************************************************
    loss, grad, H = logistic_regression(y, tx, w)
    # ***************************************************
    # INSERT YOUR CODE HERE
    # update w: TODO
    # ***************************************************
    # Search for gamma using Armijo's rule
    beta = 0.5
    sigma = 1e-2
    gamma = 1 # step size
    d = np.linalg.solve(H, grad) # direction of descent
    w_next = w - gamma*d
    loss_next = calculate_loss(y, tx, w_next)
    while (loss - loss_next) <= - sigma*gamma*grad.T.dot(d):
        gamma *= beta
        w_next = w - gamma*d
        loss_next = calculate_loss(y, tx, w_next)
    w = w_next
    return loss, w

In [8]:
def logistic_regression_newton_method_demo(y, x):
    # init parameters
    max_iter = 100
    threshold = 1e-8
    lambda_ = 0.1
    losses = []

    # build tx
    tx = np.c_[np.ones((y.shape[0], 1)), x]
    w = np.zeros((tx.shape[1], 1))
    
    # start the logistic regression
    for iter in range(max_iter-1):
        loss, w = learning_by_newton_method(y, tx, w)
        # log info
        if iter % 1 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
        # converge criterion
        losses.append(loss)
        if len(losses) > 1 and np.abs(losses[-1] - losses[-2]) < threshold:
            break
    print("loss={l}".format(l=calculate_loss(y, tx, w)))

In [ ]:
logistic_regression_newton_method_demo(y, x_reduced)